In [2]:
import pymysql, random

# 连接到MySQL数据库
conn = pymysql.connect(
    host="vultr-prod-0735f93d-5e03-4e4b-b1ae-878c590715ab-vultr-prod-7d22.vultrdb.com",     # MySQL服务器的主机名或IP地址
    user="vultradmin", # MySQL用户名
    password="AVNS_PcAfxyST1X5nxEzfYSq", # MySQL密码
    database="order",
    port=16751
)

# 创建一个游标对象，用于执行SQL查询
cursor = conn.cursor()


In [4]:
def create_order(data):
    letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    numbers = '0123456789'
    random_letters = ''.join(random.choice(letters) for _ in range(2))
    random_numbers = ''.join(random.choice(numbers) for _ in range(3))

    order_idx = '#'+random_letters+random_numbers
    total_price = 0
    price_dict = {'1':20, '2':40, '3':50, '4':30, '5':35, '6': {'1': 35, '2': 40, '3': 40, '4': 40, '5': 40}}
    for i in data:
        if i['id'] == '6':
            total_price += price_dict[i['id']][i['flavor']] * int(i['qty'])
        else:
            total_price += price_dict[i['id']] * int(i['qty'])

    # 寫入mysql dataset
    '''html格式:
    create table master
    (
        id   varchar(10)         not null,
        1_favor varchar(10)  not null,
        1_number varchar(10)  not null,
        2_favor varchar(10)  not null,
        2_number varchar(10)  not null,
        3_favor varchar(10)  not null,
        3_number varchar(10)  not null,
        4_favor varchar(10)  not null,
        4_number varchar(10)  not null,
        5_favor varchar(10)  not null,
        5_number varchar(10)  not null,
        6_favor varchar(10)  not null,
        6_number varchar(10)  not null,
        status  varchar(10)          not null,
        total varchar(10)  not null,
        primary key (id)
    );'''
    
    ordered_data = [0]*12 # 根據id填入相對應的6個品項，若該項qty為0則填入n
    for i in range(len(data)):
        ordered_data[2*int(data[i]['id'])-2] = data[i]['flavor']
        ordered_data[2*int(data[i]['id'])-1] = data[i]['qty']

    # write to sql
    prompt = '''INSERT INTO master (id, 1_favor, 1_number, 2_favor, 2_number, 3_favor, 3_number, 4_favor, 4_number, 5_favor, 5_number, 6_favor, 6_number, status, total) VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')'''.format(order_idx, ordered_data[0], ordered_data[1], ordered_data[2], ordered_data[3], ordered_data[4], ordered_data[5], ordered_data[6], ordered_data[7], ordered_data[8], ordered_data[9], ordered_data[10], ordered_data[11], '0', str(total_price))
    print(prompt)
    # cursor.execute(prompt)

    return order_idx

# 測試
data = [{'id': '6', 'flavor': '1', 'qty': '1'}]
create_order(data)

INSERT INTO master (id, 1_favor, 1_number, 2_favor, 2_number, 3_favor, 3_number, 4_favor, 4_number, 5_favor, 5_number, 6_favor, 6_number, status, total) VALUES ('#RJ862', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '35')


'#RJ862'

In [50]:
def get_all_data():
    # 從mysql中讀取data
    prompt = '''SELECT * FROM master'''
    cursor.execute(prompt)
    result = cursor.fetchall()
    return result

# 測試
(get_all_data())

(('#QY088',
  '011',
  '2',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '2',
  '40'),
 ('#RJ862',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0',
  '35'))

In [30]:
def check_order_info(target_id):
    for i in get_all_data():
        print(i[0])
        if i[0] == target_id: # id 對應到第0個數據
            print(i)
            return i[13]
    return 'error'

# 測試
check_order_info('#ZK176')

#QY088
#RJ862


'error'

In [21]:
def upload_order_info(target_id, target_state):
    # 更新訂單資訊
    prompt = '''UPDATE master SET status = '{}' WHERE id = '{}' ;'''.format(target_state, target_id)
    print(prompt)
    cursor.execute(prompt)
    conn.commit()
    
# 測試
upload_order_info('#QY088', '2')
check_order_info('#QY088')

UPDATE master SET status = '2' WHERE id = '#QY088' ;


'0'

In [46]:
def cashier_refresh(): # download orders info
    orders = list(get_all_data())
    orders = [i for i in orders if i[13] != "3"] # 排除已完成訂單
    orders = sorted(orders, key=lambda x: x[13]) # 從未付款的代號0的訂單開始排序
    
    flavor_dict = {
        '1': ['香草', '草莓', '巧克力'],
        '2': ['香草', '草莓', '巧克力'],
        '3': ['香草', '草莓', '巧克力'],
        '4': ['原味', '蒜味'],
        '5': 'n',
        '6': ['可爾必思 35元', '石榴氣泡水 35元', '葡萄氣泡水 35元','水蜜桃棄泡水 35元','蘋果氣泡水 35元']
    }
    
    result = []
    

    for o in orders:
        order_dict = {}
        order_dict['id'] = o[0]
        
        i = 0
        for item in range(1, 13, 2):
            if o[item] != 'n' and o[item+1] != '0':
                i += 1
                if len(o[item]) == 1 and o[item] != '5':
                    o_item_text = flavor_dict[o[item]][int(o[item+1])-1]
                elif o[item] == '5':
                    o_item_text = '預設'
                else:
                    o_item_text = ''
                    for w in o[item].replace(' ','').split():
                        if w == '0':
                            o_item_text += '香草'
                        elif w == '1':
                            o_item_text += '草莓'
                        elif w == '2':
                            o_item_text += '巧克力'
                order_dict[str(int((item+1)/2))] = {'flavor': o_item_text, 'qty': o[item+1]}
        order_dict['total'] = int(o[14])
        order_dict['status'] = int(o[13])
        print(order_dict)
        result.append(order_dict)

    return result

# 測試
cashier_refresh()

2
{'id': '#RJ862', '6': {'flavor': '香草', 'qty': '1'}, 'total': 35, 'status': 0}
{'id': '#QY088', '1': {'flavor': '', 'qty': '2'}, 'total': 40, 'status': 2}


[{'id': '#RJ862', '6': {'flavor': '香草', 'qty': '1'}, 'total': 35, 'status': 0},
 {'id': '#QY088', '1': {'flavor': '', 'qty': '2'}, 'total': 40, 'status': 2}]